In [1]:
%%javascript
IPython.notebook.kernel.execute('notebook = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
import os
with open(f'session.secret','r') as f:
    SESSION = f"session={f.read()}"
DAY = "16"
INPUT_FILE = f"{DAY}.txt"

In [71]:
# Manual INPUT
INPUT ="""###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

In [79]:
# Automatic INPUT
import os.path
if not os.path.isfile(INPUT_FILE):
    !curl https://adventofcode.com/2024/day/{DAY}/input --cookie {SESSION} > {INPUT_FILE}
with open(INPUT_FILE,'r') as f:
    INPUT = f.read().strip()

## Usefull import

In [148]:
import numpy as np
import pandas as pd
import networkx as nx
import copy as cp
import re

## Part 1

In [149]:
DIR = {(0,1):'>',(1,0):'v',(0,-1):'<',(-1,0):'^'}
DIRd = {v:k for k,v in DIR.items()}

In [150]:
m = [list(row.strip()) for row in INPUT.strip().split('\n') ]
G=nx.Graph()
for x,r in enumerate(m):
    for y,c in enumerate(r):
        if c=='#':
            continue
        if c=='S':
            Sx,Sy = x,y
        if c=='E':
            Ex,Ey = x,y
        
        G.add_edge((x,y,'-'),(x,y,'|'),weight=1000)
        for d,(dx,dy) in DIRd.items():
            horizontal= '-' if d in '<>' else '|'
            if m[x+dx][y+dy]!='#':
                G.add_edge((x,y,horizontal),(x+dx,y+dy,horizontal),weight=1)

In [151]:
s = nx.shortest_path_length(G,source=(Sx,Sy,'-'),weight="weight")

In [155]:
best =min(s[(Ex,Ey,'-')],s[(Ex,Ey,'|')])
best

88468

In [156]:
source=(Sx,Sy,'-')
t1 = nx.shortest_path_length(G,source=(Ex,Ey,'-'),weight="weight")
t2 = nx.shortest_path_length(G,source=(Ex,Ey,'|'),weight="weight")

In [157]:
places = set()
for g in G.nodes:
    if s[g] + t1[g]==best or s[g] + t2[g]==best:
        places.add(g[:2])
print(len(places))

616


---
---
# Incorrect solution

**This doesn't work !!**
Even if the path to a given case is shortest, turning in that case might result in higher cost than taking a.  
considering horizontal and vertical direction is a prerequisite to the problem which I did not do here

In [140]:
nei = {
    '>':[(0,1,1),(1,0,1001),(-1,0,1001),(0,-1,2001)],
    'v':[(1,0,1),(0,1,1001),(0,-1,1001),(-1,0,2001)],
    '<':[(0,-1,1),(1,0,1001),(-1,0,1001),(0,1,2001)],
    '^':[(-1,0,1),(0,1,1001),(0,-1,1001),(1,0,2001)]
}

In [141]:
m = [list(row.strip()) for row in INPUT.strip().split('\n') ]
prev = cp.deepcopy(m)

In [142]:
unvisited = set()
for x,r in enumerate(m):
    for y,c in enumerate(r):
        if c=='S':
            unvisited.add((x,y))
            m[x][y]=0
            prev[x][y]='>'
            Sx,Sy = x,y
        if c=='E':
            Ex,Ey = x,y
            m[x][y] = '.'

In [143]:
def pop_min(unvisited):
    min_d = 1e20
    mx,my= -1,-1
    for x,y in unvisited:
        if m[x][y]<min_d:
            mx=x
            my=y
            min_d=m[x][y]
    unvisited.remove((mx,my))
    return mx,my,min_d

In [144]:
while len(unvisited)>0:
    mx,my,md = pop_min(unvisited)
    dir_ = prev[mx][my]
    for dx,dy,cost in nei[dir_]:
        nx,ny = mx+dx,my+dy
        if m[nx][ny]=='#':
            continue
        if m[nx][ny]=='.' or m[nx][ny]> md+cost: ## This doesn't do
                                                 ## Even if the code is lower, turning here might result in higher cost.
                                                 ## considering horizontal and vertical direction is a prerequisite to the problem
            unvisited.add((nx,ny))
            m[nx][ny]= md+cost
            prev[nx][ny]=DIR[(dx,dy)]

In [145]:
m[Ex][Ey]

88472

In [146]:
x=Ex
y=Ey
red = "\033[31m"
black = "\033[0m"
C = 0
last_d = prev[x][y]
while x!=Sx or y!=Sy:
    C+=1
    if prev[x][y]!=last_d:
        C+=1000
        last_d = prev[x][y]
    dx,dy = DIRd[prev[x][y]]
    prev[x][y]=red+prev[x][y]+black
    x-=dx
    y-=dy
C

87472

In [117]:
for row in prev:
    print(''.join(row))

#############################################################################################################################################
#^>>>>>>>>#^>>>>>>>>>>>^#^>>>>#<<^>>#<<<<^#<<^>>>>>>#<<^#^><<<<<<<<<<^#^>>#^<<<<<^>>>>>>#<<<<<<<<^#<<^>>#<<<<<<<<<<<<<<^>>>>#<<^>^#^#^>>>^>>#
#^#######^#^###########^#^###v###^#v#v#v#^#^#^#v###v###^#^###########^#^###^#####^#######v#v###v#^###^#^#v###########^#^###^###^#^#^#^###^###
#^>>>>>>>>>>>>>>>>>>>>>>#^#<<v>>#^#v#v#v#^#<<^#v<^#v>>#^#^#<<^>>>>#^>>#<<<<<<<<<<<<<<<<<<v<v>>#v#<<<<^>>>v<<<<<^#<<<<<<^>>>>>>>>>>>>>>>>>>>>#
#^#^#v#v#####v#####v#v###^#####v#^#v###v#^###^###^###v#^#^#v#^#v###^###v#v###v#v#v#v#########v#v#####^#######^#^#v#^###^#^###v#v#^###v#####v#
#^>>#v#v#^>>#v#<<^#v>v#<<^#^>>#v>>#<<^#v#<<<<^>>>>#<<v>>#^#v#^#v#^>>#<<v#v#<<v#v#v<v>>#<<<<^#v#v>>>>#^>>>>#^#^#^#v#<<<<^#^>>>v>v<^<^>v#<<<<v#
#^###v###^#v###v#^#v#####^#^#v#######^#v#####^#v#######v#^#v#^###^###v###v#v###########v###^#######v#^###v#^#^#^#v#####^#^#######^#^###^#####
#^#^#v